In [0]:
library(foreign)
library(MASS)
library(dplyr)
library(psych)
require(car)
require(lmtest)

In [0]:
df<-read.csv(file='c:/lois/project/entireDB_2018 - for R.csv', header=T)

In [0]:
head(df)

gu,elder_ac_num,elder_hv_num,elder_lt_num,elder_po_num,elder_po_per,trans_sa_idx,elder_fpo_per,car_ro_len,car_ave_spd,road_si_num,market_num,elder_sz_num,ovp_num
강남구,351,112,267,67085,12.25,68.53,13.77,101.3,24.5,1034,10,11,20
강동구,242,106,157,58770,13.61,79.28,17.18,70.4,23.9,450,10,6,20
강북구,246,102,155,58196,18.02,76.12,21.52,34.0,21.3,219,12,2,1
강서구,210,95,133,79660,13.20,79.62,16.22,66.8,23.6,485,12,13,4
관악구,182,68,123,72249,13.89,79.78,15.84,42.3,25.3,238,16,7,16
광진구,133,48,87,45619,12.29,80.77,14.60,53.4,25.0,300,12,4,0


In [0]:
# 가로환경 개선 정책이 시행된 후 최근 도로교통공단의 연구 결과 전체 사고율은 줄었으나 노인(65세 이상) 사고율은 오히려 증가하여 자치구의 노인교통사고건수에 영향을 주는 원인을 찾고자 함
# 사고 발생의 원인과 사고 경중의 특성을 파악하기 위해 이하 3개의 모델로 나누어 분석 진행 [종속변수; 1)노인사고건수, 2)노인중상이상사고자수, 3)노인경상이하사고자수]
# 이하의 설명변수 설정 [인구통계학: 노인인구수, 노인인구비율, 노인생활인구 | 가로환경: 도로안전지표수, 시장수, 노인보호구역수 | 차로환경: 승용차차량연장, 전일평균차속 | 지표: 도로안전지수]
# 종속변수와 관련이 있는 주요 설명변수 파악을 위해,
# 개별 설명변수가 종속변수에 실제로 어느 정도의 영향을 미치는지 파악하기 위해,
# 다중회귀분석 진행 예정. 세부 모형은 이하의 방식으로 선정

In [0]:
# 본 분석에서는 관심사건(종속변수)의 발생 회수에 관심이 있으며, 각 종속변수는 count 가능한 양의 정수 데이터이므로
# 1) 이항분포, 2) 포아송분포, 3) 음이항분포 사용 가능
# 이 중 어떤 회귀모형을 사용해야 하는지는 과분산계수(VMR)로 파악 가능
# 1) 이항분포의 경우, 0<VMR<1일 때 사용 가능(분산이 평균보다 작을 때)
# 2) 포아송분포의 경우, VMR=1일 때 사용 가능(분산과 평균이 같을 때)
# 3) 음이항분포의 경우, VMR>1일 때 사용 가능(분산이 평균보다 클 때)

In [0]:
# 각 종속변수의 과분산계수 분석

In [0]:
var(df$elder_ac_num)/mean(df$elder_ac_num) # VMR(18.5692154139906) > 1

[1] 18.56922

In [0]:
var(df$elder_hv_num)/mean(df$elder_hv_num) # VMR(7.4065801056338) > 1

[1] 7.40658

In [0]:
var(df$elder_lt_num)/mean(df$elder_lt_num) # VMR(16.2335634879277) > 1

[1] 16.23356

In [0]:
# 실제 현실세계에서 발생한 교통사고 데이터의 경우, 위 3 모델 모든 경우에 분산이 평균보다 큰 과분산 문제 발생
# 따라서 분산이 평균보다 크다는 전제조건으로 시작하는 음이항 회귀모형을 세부 분석 방법으로 사용

In [0]:
df.nb<-glm.nb(elder_ac_num~elder_po_num+elder_po_per+trans_sa_idx+elder_fpo_per+car_ro_len+car_ave_spd+road_si_num+market_num+elder_sz_num+ovp_num,data=df) 
summary(df.nb);vif(df.nb)


Call:
glm.nb(formula = elder_ac_num ~ elder_po_num + elder_po_per + 
    trans_sa_idx + elder_fpo_per + car_ro_len + car_ave_spd + 
    road_si_num + market_num + elder_sz_num + ovp_num, data = df, 
    init.theta = 289.7005621, link = log)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-1.9987  -0.6088   0.1036   0.4858   2.2276  

Coefficients:
                Estimate Std. Error z value Pr(>|z|)    
(Intercept)    4.870e+00  1.318e+00   3.694 0.000221 ***
elder_po_num   6.898e-06  2.140e-06   3.223 0.001267 ** 
elder_po_per   6.078e-03  3.302e-02   0.184 0.853956    
trans_sa_idx  -2.878e-02  8.757e-03  -3.287 0.001012 ** 
elder_fpo_per  6.758e-02  2.269e-02   2.979 0.002895 ** 
car_ro_len     6.814e-04  2.731e-03   0.250 0.802949    
car_ave_spd    3.440e-02  2.302e-02   1.494 0.135064    
road_si_num    4.895e-04  2.746e-04   1.783 0.074616 .  
market_num     7.559e-03  6.082e-03   1.243 0.213909    
elder_sz_num  -1.085e-02  1.014e-02  -1.070 0.284633    
ov

elder_po_num  elder_po_per  trans_sa_idx elder_fpo_per    car_ro_len 
     3.201518      8.417293      4.552137      6.538032     10.738234 
  car_ave_spd   road_si_num    market_num  elder_sz_num       ovp_num 
     4.105244     12.869535      4.735498      2.501320      2.285469

In [0]:
# 우선적으로 1) 모델에서 음이항 회귀분석을 진행하고, 본 분석에서 선정한 설명변수가 통계적으로 유의미한지 파악하기 위해 다중공선성 검증 동시 진행
# 설명변수끼리 강한 상관관계를 가지면 다중공선성이 있음
# 애초에 설명변수끼리 종속적이라는 것 자체가 회귀분석의 가정에 위배되며, 실제로 수치적인 문제를 야기해 분석 결과를 신뢰할 수 없게 만드므로
# 본 분석에서는 vif(분산팽창인자) > 4 인 설명변수는 다중공선성이 있다고 간주하고 통계적 유의미성을 확보하기 위해 제거

In [0]:
df.nb1<-glm.nb(elder_ac_num~elder_po_num+elder_po_per+trans_sa_idx+elder_fpo_per+car_ro_len+car_ave_spd+market_num+elder_sz_num+ovp_num,data=df) 
summary(df.nb1);vif(df.nb1)


Call:
glm.nb(formula = elder_ac_num ~ elder_po_num + elder_po_per + 
    trans_sa_idx + elder_fpo_per + car_ro_len + car_ave_spd + 
    market_num + elder_sz_num + ovp_num, data = df, init.theta = 225.4475742, 
    link = log)

Deviance Residuals: 
     Min        1Q    Median        3Q       Max  
-1.85338  -0.74279  -0.02207   0.40349   2.22897  

Coefficients:
                Estimate Std. Error z value Pr(>|z|)    
(Intercept)    6.217e+00  1.133e+00   5.489 4.05e-08 ***
elder_po_num   7.037e-06  2.261e-06   3.113  0.00185 ** 
elder_po_per  -5.403e-03  3.413e-02  -0.158  0.87421    
trans_sa_idx  -3.577e-02  8.280e-03  -4.320 1.56e-05 ***
elder_fpo_per  6.241e-02  2.391e-02   2.610  0.00905 ** 
car_ro_len     4.137e-03  2.059e-03   2.010  0.04448 *  
car_ave_spd    1.310e-02  2.076e-02   0.631  0.52824    
market_num     3.854e-03  6.053e-03   0.637  0.52430    
elder_sz_num  -8.248e-03  1.066e-02  -0.774  0.43893    
ovp_num        5.784e-03  3.589e-03   1.612  0.10701    
---
Si

elder_po_num  elder_po_per  trans_sa_idx elder_fpo_per    car_ro_len 
     3.208324      7.948749      3.563442      6.393337      5.337589 
  car_ave_spd    market_num  elder_sz_num       ovp_num 
     2.975278      4.190637      2.448175      2.215894

In [0]:
df.nb2<-glm.nb(elder_ac_num~elder_po_num+trans_sa_idx+elder_fpo_per+car_ro_len+car_ave_spd+market_num+elder_sz_num+ovp_num,data=df) 
summary(df.nb2);vif(df.nb2)


Call:
glm.nb(formula = elder_ac_num ~ elder_po_num + trans_sa_idx + 
    elder_fpo_per + car_ro_len + car_ave_spd + market_num + elder_sz_num + 
    ovp_num, data = df, init.theta = 223.7845305, link = log)

Deviance Residuals: 
     Min        1Q    Median        3Q       Max  
-1.86106  -0.74194  -0.07382   0.42749   2.24224  

Coefficients:
                Estimate Std. Error z value Pr(>|z|)    
(Intercept)    6.149e+00  1.050e+00   5.856 4.74e-09 ***
elder_po_num   7.197e-06  2.021e-06   3.562 0.000368 ***
trans_sa_idx  -3.555e-02  8.164e-03  -4.354 1.34e-05 ***
elder_fpo_per  5.956e-02  1.548e-02   3.847 0.000120 ***
car_ro_len     4.231e-03  1.973e-03   2.144 0.032042 *  
car_ave_spd    1.346e-02  2.070e-02   0.650 0.515588    
market_num     3.637e-03  5.906e-03   0.616 0.537954    
elder_sz_num  -8.177e-03  1.067e-02  -0.766 0.443435    
ovp_num        5.814e-03  3.593e-03   1.618 0.105662    
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

(Dispersion par

elder_po_num  trans_sa_idx elder_fpo_per    car_ro_len   car_ave_spd 
     2.554724      3.449776      2.670620      4.884208      2.946133 
   market_num  elder_sz_num       ovp_num 
     3.975492      2.444535      2.212186

In [0]:
df.nb3<-glm.nb(elder_ac_num~elder_po_num+trans_sa_idx+elder_fpo_per+car_ave_spd+market_num+elder_sz_num+ovp_num,data=df) 
summary(df.nb3);vif(df.nb3)


Call:
glm.nb(formula = elder_ac_num ~ elder_po_num + trans_sa_idx + 
    elder_fpo_per + car_ave_spd + market_num + elder_sz_num + 
    ovp_num, data = df, init.theta = 160.669801, link = log)

Deviance Residuals: 
     Min        1Q    Median        3Q       Max  
-2.01634  -0.85313  -0.00511   0.54383   2.14555  

Coefficients:
                Estimate Std. Error z value Pr(>|z|)    
(Intercept)    7.746e+00  8.004e-01   9.678  < 2e-16 ***
elder_po_num   7.067e-06  2.199e-06   3.214  0.00131 ** 
trans_sa_idx  -4.790e-02  6.298e-03  -7.604 2.86e-14 ***
elder_fpo_per  4.294e-02  1.460e-02   2.942  0.00326 ** 
car_ave_spd    9.752e-03  2.241e-02   0.435  0.66341    
market_num    -2.839e-03  5.501e-03  -0.516  0.60587    
elder_sz_num   1.957e-03  1.047e-02   0.187  0.85165    
ovp_num        5.762e-03  3.944e-03   1.461  0.14400    
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

(Dispersion parameter for Negative Binomial(160.6698) family taken to be 1)

    Null 

elder_po_num  trans_sa_idx elder_fpo_per   car_ave_spd    market_num 
     2.542440      1.689537      1.976692      2.903326      2.900451 
 elder_sz_num       ovp_num 
     1.965468      2.189604

In [0]:
# 다중공선성이 있는 설명변수는 제거한 후, 각 모델별 음이항 회귀분석 진행. 1)노인사고건수, 2)노인중상이상사고자수, 3)노인경상이하사고자수

In [0]:
df_ac<-df.nb3
summary(df_ac);vif(df_ac)


Call:
glm.nb(formula = elder_ac_num ~ elder_po_num + trans_sa_idx + 
    elder_fpo_per + car_ave_spd + market_num + elder_sz_num + 
    ovp_num, data = df, init.theta = 160.669801, link = log)

Deviance Residuals: 
     Min        1Q    Median        3Q       Max  
-2.01634  -0.85313  -0.00511   0.54383   2.14555  

Coefficients:
                Estimate Std. Error z value Pr(>|z|)    
(Intercept)    7.746e+00  8.004e-01   9.678  < 2e-16 ***
elder_po_num   7.067e-06  2.199e-06   3.214  0.00131 ** 
trans_sa_idx  -4.790e-02  6.298e-03  -7.604 2.86e-14 ***
elder_fpo_per  4.294e-02  1.460e-02   2.942  0.00326 ** 
car_ave_spd    9.752e-03  2.241e-02   0.435  0.66341    
market_num    -2.839e-03  5.501e-03  -0.516  0.60587    
elder_sz_num   1.957e-03  1.047e-02   0.187  0.85165    
ovp_num        5.762e-03  3.944e-03   1.461  0.14400    
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

(Dispersion parameter for Negative Binomial(160.6698) family taken to be 1)

    Null 

elder_po_num  trans_sa_idx elder_fpo_per   car_ave_spd    market_num 
     2.542440      1.689537      1.976692      2.903326      2.900451 
 elder_sz_num       ovp_num 
     1.965468      2.189604

In [0]:
df_hv<-glm.nb(elder_hv_num~elder_po_num+trans_sa_idx+elder_fpo_per+car_ave_spd+market_num+elder_sz_num+ovp_num,data=df) 
summary(df_hv);vif(df_hv)


Call:
glm.nb(formula = elder_hv_num ~ elder_po_num + trans_sa_idx + 
    elder_fpo_per + car_ave_spd + market_num + elder_sz_num + 
    ovp_num, data = df, init.theta = 123.0087214, link = log)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.2483  -0.6138  -0.2910   0.5938   2.5612  

Coefficients:
                Estimate Std. Error z value Pr(>|z|)    
(Intercept)    5.634e+00  1.072e+00   5.257 1.47e-07 ***
elder_po_num   9.754e-06  2.983e-06   3.269  0.00108 ** 
trans_sa_idx  -3.481e-02  8.456e-03  -4.117 3.84e-05 ***
elder_fpo_per  6.225e-02  1.967e-02   3.165  0.00155 ** 
car_ave_spd   -6.264e-03  3.035e-02  -0.206  0.83651    
market_num     3.373e-03  7.370e-03   0.458  0.64716    
elder_sz_num   2.144e-03  1.401e-02   0.153  0.87841    
ovp_num        6.040e-03  5.379e-03   1.123  0.26150    
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

(Dispersion parameter for Negative Binomial(123.0087) family taken to be 1)

    Null deviance:

elder_po_num  trans_sa_idx elder_fpo_per   car_ave_spd    market_num 
     2.551382      1.677109      2.046493      2.931280      2.822552 
 elder_sz_num       ovp_num 
     1.987478      2.226557

In [0]:
df_lt<-glm.nb(elder_lt_num~elder_po_num+trans_sa_idx+elder_fpo_per+car_ave_spd+market_num+elder_sz_num+ovp_num,data=df) 
summary(df_lt);vif(df_lt)


Call:
glm.nb(formula = elder_lt_num ~ elder_po_num + trans_sa_idx + 
    elder_fpo_per + car_ave_spd + market_num + elder_sz_num + 
    ovp_num, data = df, init.theta = 117.5879037, link = log)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-1.5666  -0.7939  -0.1344   0.6348   1.8129  

Coefficients:
                Estimate Std. Error z value Pr(>|z|)    
(Intercept)    8.131e+00  9.529e-01   8.533  < 2e-16 ***
elder_po_num   4.590e-06  2.625e-06   1.749   0.0804 .  
trans_sa_idx  -5.672e-02  7.483e-03  -7.580 3.45e-14 ***
elder_fpo_per  3.266e-02  1.746e-02   1.871   0.0613 .  
car_ave_spd    1.944e-02  2.673e-02   0.727   0.4671    
market_num    -8.243e-03  6.588e-03  -1.251   0.2108    
elder_sz_num   7.435e-03  1.249e-02   0.595   0.5517    
ovp_num        6.114e-03  4.689e-03   1.304   0.1923    
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

(Dispersion parameter for Negative Binomial(117.5879) family taken to be 1)

    Null deviance:

elder_po_num  trans_sa_idx elder_fpo_per   car_ave_spd    market_num 
     2.531381      1.700983      1.953142      2.877780      2.888803 
 elder_sz_num       ovp_num 
     1.964747      2.205195

In [0]:
# p값(Pr(>/z/)이 작다는 것은 회귀계수가 유의하다는 뜻이고, 값이 작을수록 상관관계가 있다는 것을 나타냄
# 이 값이 얼마나 작은지에 따라 바로 아래에 있는 Signif. codes대로 점이 찍히게 됨
# 보통 유의수준은 5%로 잡으므로, 점 하나만 찍혀도 상관관계는 있다고 봐도 무방
# 만약 이 값이 커서 회귀관계가 없는 것으로 밝혀진다면 회귀계수가 어떻게 구해지든 통계적으로는 의미가 없으므로
# 각 모델의 분석 결과를 바탕으로 통계적으로 유의미하다고 볼 수 있는 설명변수들만 가지고 재분석

In [0]:
df_ac_fit<-glm.nb(elder_ac_num~elder_po_num+trans_sa_idx+elder_fpo_per,data=df)
summary(df_ac_fit);vif(df_ac_fit);confint(df_ac_fit)


Call:
glm.nb(formula = elder_ac_num ~ elder_po_num + trans_sa_idx + 
    elder_fpo_per, data = df, init.theta = 102.6570633, link = log)

Deviance Residuals: 
     Min        1Q    Median        3Q       Max  
-2.01076  -0.66569  -0.05484   0.53726   1.83329  

Coefficients:
                Estimate Std. Error z value Pr(>|z|)    
(Intercept)    8.076e+00  4.528e-01  17.837  < 2e-16 ***
elder_po_num   8.606e-06  1.625e-06   5.298 1.17e-07 ***
trans_sa_idx  -4.652e-02  5.873e-03  -7.921 2.36e-15 ***
elder_fpo_per  2.714e-02  1.280e-02   2.120    0.034 *  
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

(Dispersion parameter for Negative Binomial(102.6571) family taken to be 1)

    Null deviance: 130.973  on 24  degrees of freedom
Residual deviance:  25.322  on 21  degrees of freedom
AIC: 245.38

Number of Fisher Scoring iterations: 1


              Theta:  102.7 
          Std. Err.:  43.0 

 2 x log-likelihood:  -235.376 

elder_po_num  trans_sa_idx elder_fpo_per 
     1.065131      1.089143      1.144391

Waiting for profiling to be done...


,2.5 %,97.5 %
(Intercept),7.182646e+00,8.979221e+00
elder_po_num,5.344386e-06,1.186766e-05
trans_sa_idx,-5.814479e-02,-3.498323e-02
elder_fpo_per,1.827910e-03,5.261019e-02


In [0]:
df_hv_fit<-glm.nb(elder_hv_num~elder_po_num+trans_sa_idx+elder_fpo_per,data=df)
summary(df_hv_fit);vif(df_hv_fit);confint(df_hv_fit)


Call:
glm.nb(formula = elder_hv_num ~ elder_po_num + trans_sa_idx + 
    elder_fpo_per, data = df, init.theta = 110.4902327, link = log)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-1.8173  -0.7436  -0.2346   0.5019   2.5729  

Coefficients:
                Estimate Std. Error z value Pr(>|z|)    
(Intercept)    6.019e+00  5.373e-01  11.204  < 2e-16 ***
elder_po_num   9.369e-06  1.960e-06   4.779 1.76e-06 ***
trans_sa_idx  -3.855e-02  7.012e-03  -5.498 3.85e-08 ***
elder_fpo_per  5.531e-02  1.504e-02   3.677 0.000236 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

(Dispersion parameter for Negative Binomial(110.4902) family taken to be 1)

    Null deviance: 95.839  on 24  degrees of freedom
Residual deviance: 24.432  on 21  degrees of freedom
AIC: 207.02

Number of Fisher Scoring iterations: 1


              Theta:  110.5 
          Std. Err.:  67.9 

 2 x log-likelihood:  -197.018 

elder_po_num  trans_sa_idx elder_fpo_per 
     1.055621      1.100283      1.145484

Waiting for profiling to be done...


,2.5 %,97.5 %
(Intercept),4.956559e+00,7.082706e+00
elder_po_num,5.520988e-06,1.323238e-05
trans_sa_idx,-5.241871e-02,-2.471162e-02
elder_fpo_per,2.535239e-02,8.532896e-02


In [0]:
df_lt_fit<-glm.nb(elder_lt_num~elder_po_num+trans_sa_idx+elder_fpo_per,data=df)
summary(df_lt_fit);vif(df_lt_fit);confint(df_lt_fit)


Call:
glm.nb(formula = elder_lt_num ~ elder_po_num + trans_sa_idx + 
    elder_fpo_per, data = df, init.theta = 55.41750206, link = log)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.4804  -0.4801  -0.1974   0.6566   2.1216  

Coefficients:
                Estimate Std. Error z value Pr(>|z|)    
(Intercept)    8.399e+00  5.978e-01  14.049  < 2e-16 ***
elder_po_num   8.217e-06  2.140e-06   3.840 0.000123 ***
trans_sa_idx  -5.132e-02  7.745e-03  -6.626 3.44e-11 ***
elder_fpo_per  6.714e-03  1.700e-02   0.395 0.692867    
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

(Dispersion parameter for Negative Binomial(55.4175) family taken to be 1)

    Null deviance: 94.202  on 24  degrees of freedom
Residual deviance: 25.325  on 21  degrees of freedom
AIC: 238.6

Number of Fisher Scoring iterations: 1


              Theta:  55.4 
          Std. Err.:  21.8 

 2 x log-likelihood:  -228.597 

elder_po_num  trans_sa_idx elder_fpo_per 
     1.067173      1.087418      1.144642

Waiting for profiling to be done...


,2.5 %,97.5 %
(Intercept),7.214198e+00,9.6014578956
elder_po_num,3.867594e-06,0.0000125601
trans_sa_idx,-6.673115e-02,-0.0360841026
elder_fpo_per,-2.664889e-02,0.0403801136


In [0]:
# 본 분석모델의 타당성을 검증하기 위해 우도비(likelyhood) 검정 실행 예정